In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from torch.autograd import Variable
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score

DATA_DIR = 'Data/'

In [ ]:
student_test_logs = pd.read_pickle(DATA_DIR + 'student_test_logs')
student_train_logs = pd.read_pickle(DATA_DIR + 'student_train_logs')
print(student_train_logs.shape)
student_train_logs.head()

In [ ]:
train_labels = pd.read_csv('Data/training_label.csv', index_col='ITEST_id').sort_index()
train_labels.drop_duplicates(subset=None, keep='first', inplace=True)
train_labels.head()

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, fixed_dim, output_dim, batch_size=1, n_layers=1):
        super(LSTMClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=n_layers,
                            dropout=0.25,
                            bidirectional=True)
        self.hidden2label = nn.Linear(hidden_dim*2 + fixed_dim, output_dim)

    def init_hidden(self):
        h0 = Variable(torch.zeros(self.n_layers*2, self.batch_size, self.hidden_dim))
        c0 = Variable(torch.zeros(self.n_layers*2, self.batch_size, self.hidden_dim))
        return (h0, c0)

    def forward(self, actions, fixed):
        hidden_state = self.init_hidden()
        out, _ = self.lstm(actions, hidden_state)
        y = self.hidden2label(torch.cat([out[-1, :, :], fixed], dim=1))
        return y

In [ ]:
class TrainingSet(Dataset):
    def __init__(self):
        self.actions = student_train_logs.drop(['SY ASSISTments Usage',
                                        'AveKnow',
                                        'AveCarelessness',
                                        'AveCorrect',
                                        'NumActions',
                                        'AveResBored',
                                        'AveResEngcon',
                                        'AveResConf',
                                        'AveResFrust',
                                        'AveResOfftask',
                                        'AveResGaming'], axis=1)
        
        self.fixed = student_train_logs[['ITEST_id',
                                        'SY ASSISTments Usage',
                                        'AveKnow',
                                        'AveCarelessness',
                                        'AveCorrect',
                                        'NumActions',
                                        'AveResBored',
                                        'AveResEngcon',
                                        'AveResConf',
                                        'AveResFrust',
                                        'AveResOfftask',
                                        'AveResGaming']]
                
        self.y = train_labels

        self.weights = [0.7, 0.3]
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, id):
        studentId = self.y.iloc[id].name
        actions = self.actions[self.actions['ITEST_id'] == studentId].as_matrix().astype(np.float32)
        
        fixed_df = self.fixed[self.fixed['ITEST_id'] == studentId].assign(MCAS=self.y.loc[studentId].MCAS, SchoolId=self.y.loc[studentId].SchoolId)
        fixed = fixed_df.as_matrix().astype(np.float32)[0]
        
        target = np.asarray([self.y.loc[studentId].isSTEM]).astype(np.float32)
        
        return torch.from_numpy(actions), torch.from_numpy(fixed), torch.from_numpy(target)

In [ ]:
train_dataset = TrainingSet()
sampler = WeightedRandomSampler(train_dataset.weights, num_samples=len(train_dataset))
train_loader = DataLoader(train_dataset, sampler=sampler, batch_size=1, num_workers=4)

n_hidden = 32
lstm = LSTMClassifier(train_dataset.actions.shape[1] - 1,
                      n_hidden,
                      train_dataset.fixed.shape[1] - 1 + 2, # + 2 for MCAS and SchoolId
                      output_dim=1,
                      batch_size=1,
                      n_layers=1)

criterion = nn.BCEWithLogitsLoss()
learning_rate = 0.001 # If you set this too high, it might explode. If too low, it might not learn
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)

losses = []
accs = []
bar1 = tqdm_notebook(range(5))
for epoch in bar1:
    train_loss = 0
    train_acc = []
    
    for i, (actions, fixed, target) in enumerate(tqdm_notebook(train_loader)):
        actions = Variable(actions[:, :, 1:]).permute(1,0,2)
        fixed = Variable(fixed[:, 1:])
        target = Variable(target)

        lstm.zero_grad()
        output = lstm(actions, fixed)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.data[0]
        
        _, argmax = output.data.max(1)
        y_preds = argmax.view(-1).numpy()
        y_true = target.data.view(-1).numpy()

        train_acc.append(accuracy_score(y_true, y_preds))

    train_loss /= i+1
    
    losses.append(train_loss)
    accs.append(np.mean(train_acc))
    
    bar1.set_postfix(loss=losses[-1], acc=accs[-1])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.plot(accs)
plt.show()